In [71]:
import duckdb

# Create an in-memory DuckDB connection
con = duckdb.connect(database=':memory:')

# Register logins.csv
con.execute("""
    CREATE OR REPLACE TEMP VIEW logins AS
    SELECT * FROM read_csv_auto('data/logins.csv')
""")

# Register transactions.csv
con.execute("""
    CREATE OR REPLACE TEMP VIEW transactions AS
    SELECT * FROM read_csv_auto('data/transactions.csv')
""")

# Register chargebacks.csv
con.execute("""
    CREATE OR REPLACE TEMP VIEW chargebacks AS
    SELECT * FROM read_csv_auto('data/chargebacks.csv')
""")

In [73]:
import pandas as pd

logins = pd.read_csv('data/logins.csv')
chargebacks = pd.read_csv('data/chargebacks.csv')
transactions = pd.read_csv('data/transactions.csv')


In [74]:
logins.head(2)

,user_id,timestamp,ip,device_id,outcome
0,147,2024-01-17 10:18:25,192.168.177.150,device_28,success
1,126,2024-02-10 18:05:31,192.168.171.37,device_5,fail


In [75]:
chargebacks.head(2)

,transaction_id,user_id,amount,days_after_purchase
0,60,159,62.81,27
1,1242,183,40.96,7


In [76]:
transactions.head(2)

,transaction_id,user_id,timestamp,amount,payment_method,is_fraud
0,1,197,2024-01-31 04:02:59,100.75,card,0
1,2,198,2024-01-25 01:27:10,202.50,paypal,0


# Level 1 Basics

In [78]:
## 1. Count the total number of failed logins across all user
sql_1 = duckdb.sql("""
    SELECT COUNT(*)
    FROM logins
    WHERE outcome = 'fail'
""").df()
print('1. Total number of failed logins across all users: '+ str(sql_1['count_star()'][0]))

## 2. Find the top 5 users with the highest number of failed logins
sql_2 = duckdb.sql("""
    SELECT
        user_id,
        COUNT(*) AS failed_login_count
    FROM logins
    WHERE outcome = 'fail'
    GROUP BY user_id
    ORDER BY failed_login_count DESC
    LIMIT 5;
""").df()
print('2. Top 5 users with the highest number of failed logins: '+ str(sql_2['user_id'].to_list()))

## 3. For each payment method, compute the total number of transactions and fraud rate

sql_3 = duckdb.sql("""
    SELECT
        payment_method,
        COUNT(*)                                            AS total_transactions,
        SUM(is_fraud)                                      AS total_fraud_transactions,
        ROUND(100.0 * SUM(is_fraud)::DOUBLE / COUNT(*), 2) AS fraud_rate_pct
    FROM transactions
    GROUP BY payment_method
    ORDER BY fraud_rate_pct DESC;
""").df()
sql_3

1. Total number of failed logins across all users: 207
2. Top 5 users with the highest number of failed logins: [90, 37, 29, 148, 26]


,payment_method,total_transactions,total_fraud_transactions,fraud_rate_pct
0,paypal,308,16.0,5.19
1,crypto,140,7.0,5.00
2,card,902,44.0,4.88
3,voucher,150,5.0,3.33


# Level 2: Pattern Detection

In [ ]:
## 4. Identify users with more than 5 failed logins within any single hour.
sql_4 = duckdb.sql("""
    SELECT 
        user_id, 
        DATE_TRUNC('hour', CAST(timestamp AS TIMESTAMP)) as ts_hour,
        COUNT(*) as failed_login_count
    FROM logins
    WHERE outcome = 'fail'
    GROUP BY user_id, ts_hour
    HAVING COUNT(*) > 5
    ORDER BY failed_login_count DESC
""").df()

## 5. Calculate the weekly fraud rate (fraudulent transactions ÷ total transactions).
sql_5 = duckdb.sql("""
    SELECT 
        DATE_TRUNC('week', CAST(timestamp AS TIMESTAMP)) as ts_week,
        COUNT(*) as total_weekly_transactions,
        SUM(is_fraud) as total_weekly_fraud_transactions,
        ROUND(100.0 * SUM(is_fraud)::DOUBLE / COUNT(*), 2) as weekly_fraud_rate_pct
    FROM transactions
    GROUP BY ts_week
    ORDER BY ts_week ASC
""").df()

## 6. List the top 10 users by total transaction amount, and mark how many were fraud.
sql_6 = duckdb.sql("""
    SELECT 
        user_id,
        COUNT(*) as total_transactions,
        SUM(amount) as total_amount,
        SUM(is_fraud) as total_fraud_transactions,
        SUM(CASE
            WHEN is_fraud = 1 THEN amount
            ELSE 0 
            END
        ) as total_fraud_amount,
        ROUND(100.0 * SUM(is_fraud)::DOUBLE / NULLIF(COUNT(*),0), 2) AS fraud_tx_share_pct
    FROM transactions
    GROUP BY user_id
    ORDER BY total_amount DESC
    LIMIT 10;
""").df()

# sql_4
# sql_5
# sql_6

# Level 3 Deeper Fraud Insights

In [129]:
## 7. Find users who had fraudulent transactions AND chargebacks.
sql_7 = duckdb.sql("""
    SELECT 
        user_id
    FROM chargebacks
    WHERE user_id in (SELECT DISTINCT (user_id) as user_id
        FROM transactions
        WHERE is_fraud = 1
    )
""").df()

## 8. Detect suspicious login behavior: users who logged in from more than 3 distinct IPs within the same day.
sql_8 = duckdb.sql("""
    SELECT 
        user_id,
        DATE_TRUNC('day', CAST(timestamp AS TIMESTAMP)) as ts_day,
        COUNT(DISTINCT ip) as unique_ip_count_per_day
    FROM logins
    GROUP BY user_id, ts_day
    HAVING COUNT(DISTINCT ip) > 3
    ORDER BY unique_ip_count_per_day DESC;
""").df()

## 9. Calculate the average chargeback amount by days_after_purchase bucket: 0–10 days, 11–30 days, 31–60 days.
sql_9 = duckdb.sql("""
    SELECT 
        CASE 
            WHEN days_after_purchase BETWEEN 0 AND 10 THEN '0-10 days'
            WHEN days_after_purchase BETWEEN 11 AND 30 THEN '11-30 days'
            WHEN days_after_purchase BETWEEN 31 AND 60 THEN '31-60 days'
            ELSE '> 60 days'
        END as days_after_purchase_bucket,
        COUNT(*) as chargeback_count,
        ROUND(AVG(amount),2) as average_chargeback_by_days_bucket
    FROM chargebacks
    GROUP BY days_after_purchase_bucket
""").df()
sql_9

,days_after_purchase_bucket,chargeback_count,average_chargeback_by_days_bucket
0,0-10 days,7,36.06
1,11-30 days,14,41.59
2,31-60 days,15,58.76


In [130]:
con.execute("DROP VIEW IF EXISTS logins")